In [6]:
from xopto.mcml import mc
from xopto.cl import clinfo

import numpy as np
from matplotlib import pyplot as plt
import pickle
from itertools import product
from tqdm import tqdm
from decimal import Decimal, Context


In [2]:
c = Context(prec=5)
mu_as = [c.create_decimal_from_float(x) for x in np.logspace(-2.4,0,150)]
mu_ss = [c.create_decimal_from_float(x) for x in np.logspace(1.8,2,100)]
mua_mus_pairs = list(product(np.array(mu_as),
                            np.array(mu_ss)))

In [7]:
#trace = mc.mctrace.Trace(128)
source = source = mc.mcsource.Line(
    position=(0.0, 0.0, 0.0),
    direction=(0.0, 0.0, 1.0)
)


# creating surface detectors of backscattered or transmitted light
detectors = mc.mcdetector.Detectors(
    top = mc.mcdetector.Radial(
        mc.mcdetector.RadialAxis(0.0, 20.0e-3, 100, logscale=False)
    )
    
)


# selecting the first available OpenCL GPU device
gpu = clinfo.gpu()


In [11]:
i = 0
res = []

for mua,mus in tqdm(mua_mus_pairs[:]):
    layers = mc.mclayer.Layers(
        [
            mc.mclayer.Layer(n=1.0, d=0,  mua=0.0e2, mus=0.0e2, pf=mc.mcpf.Hg(0.0)),
            #mc.mclayer.Layer(n=1.47, d=0.40e-3, mua=0.0*10, mus=0.0, pf=mc.mcpf.Hg(0.9)),
            mc.mclayer.Layer(n=1.33, d=30.0e-3, mua=mua*1000, mus=mus*1000, pf=mc.mcpf.Hg(0.9)),
            mc.mclayer.Layer(n=1.00, d=np.inf,  mua=0.0e2, mus=0.0e2, pf=mc.mcpf.Hg(0.0))
        ]
    )
    mc_obj = mc.Mc(layers, source, detectors, cl_devices=gpu)
    mc_obj.rmax = 10e-2


# running the MC simulation with 10,000,000 photon packets
    trace_res_nail, fluence_res, detectors_res_nail = mc_obj.run(1e7)
    res.append((mua,mus,detectors_res_nail.top.reflectance))
    if i%1000 == 0:
        #print(i)
        with open(f'res2_{i}.pkl', 'wb') as f:
            pickle.dump(res, f)
        
    #df.loc[i]=[mua,mus,*(detectors_res_nail.top.reflectance)]
    i += 1

  0%|          | 0/15000 [00:02<?, ?it/s]


KeyboardInterrupt: 